In [2]:
import os
import pandas as pd
pd.set_option('display.max_columns', 500)
DATA_INPUT_PATH = "../data/inputs"
DATA_OUTPUT_PATH = "../data/outputs"

In [3]:
data = pd.read_csv(os.path.join(DATA_INPUT_PATH, "nba_logreg.csv"))

In [4]:
data.head()

,Name,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
0,Brandon Ingram,36,27.4,7.4,2.6,7.6,34.7,0.5,2.1,25.0,1.6,2.3,69.9,0.7,3.4,4.1,1.9,0.4,0.4,1.3,0.0
1,Andrew Harrison,35,26.9,7.2,2.0,6.7,29.6,0.7,2.8,23.5,2.6,3.4,76.5,0.5,2.0,2.4,3.7,1.1,0.5,1.6,0.0
2,JaKarr Sampson,74,15.3,5.2,2.0,4.7,42.2,0.4,1.7,24.4,0.9,1.3,67.0,0.5,1.7,2.2,1.0,0.5,0.3,1.0,0.0
3,Malik Sealy,58,11.6,5.7,2.3,5.5,42.6,0.1,0.5,22.6,0.9,1.3,68.9,1.0,0.9,1.9,0.8,0.6,0.1,1.0,1.0
4,Matt Geiger,48,11.5,4.5,1.6,3.0,52.4,0.0,0.1,0.0,1.3,1.9,67.4,1.0,1.5,2.5,0.3,0.3,0.4,0.8,1.0


In [5]:
data.describe()

,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
count,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1329.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000
mean,60.414179,17.624627,6.801493,2.629104,5.885299,44.169403,0.247612,0.779179,19.308126,1.297687,1.821940,70.300299,1.009403,2.025746,3.034478,1.550522,0.618507,0.368582,1.193582,0.620149
std,17.433992,8.307964,4.357545,1.683555,3.593488,6.137679,0.383688,1.061847,16.022916,0.987246,1.322984,10.578479,0.777119,1.360008,2.057774,1.471169,0.409759,0.429049,0.722541,0.485531
min,11.000000,3.100000,0.700000,0.300000,0.800000,23.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.300000,0.000000,0.000000,0.000000,0.100000,0.000000
25%,47.000000,10.875000,3.700000,1.400000,3.300000,40.200000,0.000000,0.000000,0.000000,0.600000,0.900000,64.700000,0.400000,1.000000,1.500000,0.600000,0.300000,0.100000,0.700000,0.000000
50%,63.000000,16.100000,5.550000,2.100000,4.800000,44.100000,0.100000,0.300000,22.400000,1.000000,1.500000,71.250000,0.800000,1.700000,2.500000,1.100000,0.500000,0.200000,1.000000,1.000000
75%,77.000000,22.900000,8.800000,3.400000,7.500000,47.900000,0.400000,1.200000,32.500000,1.600000,2.300000,77.600000,1.400000,2.600000,4.000000,2.000000,0.800000,0.500000,1.500000,1.000000
max,82.000000,40.900000,28.200000,10.200000,19.800000,73.700000,2.300000,6.500000,100.000000,7.700000,10.200000,100.000000,5.300000,9.600000,13.900000,10.600000,2.500000,3.900000,4.400000,1.000000


In [6]:
def find_duplicate_names(data):
  return set([x for x in data["Name"] if data["Name"].to_list().count(x) > 1])

In [7]:
find_duplicate_names(data)

{'Bob Martin',
 'Bobby Jones',
 'Carlos Rogers',
 'Cedric Henderson',
 'Charles Jones',
 'Charles Smith',
 'Chris Smith',
 'Cory Alexander',
 'David Lee',
 'Dee Brown',
 'Eddie Johnson',
 'Gerald Henderson',
 'Glen Rice',
 'Glenn Robinson',
 'James Johnson',
 'Jamie Feick',
 'Jeff Taylor',
 'Ken Johnson',
 'Larry Drew',
 'Larry Johnson',
 'Marcus Williams',
 'Mark Bryant',
 'Mark Davis',
 'Michael Anderson',
 'Michael Smith',
 'Mike Dunleavy',
 'Reggie Williams',
 'Tim Hardaway',
 'Walker Russell'}

In [60]:
data[data["Name"]=="Larry Johnson"]

,Name,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
18,Larry Johnson,82,37.2,19.2,7.5,15.3,49.0,0.1,0.3,22.7,4.1,5.0,82.9,3.9,7.0,11.0,3.6,1.0,0.6,1.9,0.0
19,Larry Johnson,82,37.2,19.2,7.5,15.3,49.0,0.1,0.3,22.7,4.1,5.0,82.9,3.9,7.0,11.0,3.6,1.0,0.6,1.9,1.0


In [62]:
data

,Name,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
0,Brandon Ingram,36,27.4,7.4,2.6,7.6,34.7,0.5,2.1,25.0,1.6,2.3,69.9,0.7,3.4,4.1,1.9,0.4,0.4,1.3,0.0
1,Andrew Harrison,35,26.9,7.2,2.0,6.7,29.6,0.7,2.8,23.5,2.6,3.4,76.5,0.5,2.0,2.4,3.7,1.1,0.5,1.6,0.0
2,JaKarr Sampson,74,15.3,5.2,2.0,4.7,42.2,0.4,1.7,24.4,0.9,1.3,67.0,0.5,1.7,2.2,1.0,0.5,0.3,1.0,0.0
3,Malik Sealy,58,11.6,5.7,2.3,5.5,42.6,0.1,0.5,22.6,0.9,1.3,68.9,1.0,0.9,1.9,0.8,0.6,0.1,1.0,1.0
4,Matt Geiger,48,11.5,4.5,1.6,3.0,52.4,0.0,0.1,0.0,1.3,1.9,67.4,1.0,1.5,2.5,0.3,0.3,0.4,0.8,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335,Chris Smith,80,15.8,4.3,1.6,3.6,43.3,0.0,0.2,14.3,1.2,1.5,79.2,0.4,0.8,1.2,2.5,0.6,0.2,0.8,0.0
1336,Brent Price,68,12.6,3.9,1.5,4.1,35.8,0.1,0.7,16.7,0.8,1.0,79.4,0.4,1.1,1.5,2.3,0.8,0.0,1.3,1.0
1337,Marlon Maxey,43,12.1,5.4,2.2,3.9,55.0,0.0,0.0,0.0,1.0,1.6,64.3,1.5,2.3,3.8,0.3,0.3,0.4,0.9,0.0
1338,Litterial Green,52,12.0,4.5,1.7,3.8,43.9,0.0,0.2,10.0,1.2,1.8,62.5,0.2,0.4,0.7,2.2,0.4,0.1,0.8,1.0


In [70]:
from ydata_profiling import ProfileReport

In [72]:
profile = ProfileReport(data, title="NBA Player Statistics")
profile.to_file(os.path.join(DATA_OUTPUT_PATH, "nba_player_report.html")) 

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [73]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=3,random_state=50,shuffle=True)

In [79]:
for trainnig_id, test_id in kf.split(data):
    print(len(trainnig_id), len(test_id))

893 447
893 447
894 446


In [10]:
data[data["TARGET_5Yrs"]==1].Name.to_numpy()

array(['Malik Sealy', 'Matt Geiger', 'Don MacLean', 'Tracy Murray',
       'Sam Mack', 'Lorenzo Williams', 'Elmore Spencer', 'John Crotty',
       'Larry Johnson', 'Billy Owens', 'Stacey Augmon', 'Mark Macon',
       'Larry Stewart', 'Doug Smith', 'Donald Hodge', 'Dale Davis',
       'Stanley Roberts', 'Terrell Brandon', 'Bison Dele', 'Tracy Moore',
       'Greg Anthony', 'Kenny Anderson', 'Victor Alexander',
       'David Benoit', 'Robert Pack', 'Pete Chilcutt', 'Chris Gatling',
       'Eric Murdock', 'Randy Brown', 'Elliot Perry', 'Marty Conlon',
       'Lionel Simmons', 'Derrick Coleman', 'Dennis Scott',
       'Gary Payton*', 'Felton Spencer', 'Willie Burton', 'Dee Brown',
       'Dee Brown', 'Kendall Gill', 'Duane Causwell',
       'Mahmoud Abdul-Rauf', 'Danny Ferry', 'Mario Elie', 'Bimbo Coles',
       'Loy Vaught', 'Tyrone Hill', 'Terry Mills', 'Henry James',
       'Larry Robinson', 'Jud Buechler', 'Cedric Ceballos', 'Greg Foster',
       'Tony Smith', 'Steve Henson', 'Jayson W

In [21]:
import re
[name for name in data[data["TARGET_5Yrs"]==1].Name.to_numpy() if re.search("smith", name, flags=re.IGNORECASE)]

['Doug Smith',
 'Tony Smith',
 'Michael Smith',
 'Michael Smith',
 'Charles Smith',
 'Charles Smith',
 'Charles Smith',
 'Charles Smith',
 'Charles Smith',
 'Charles Smith',
 'Kenny Smith',
 'Larry Smith',
 'Ish Smith',
 'Jason Smith',
 'Craig Smith',
 'Josh Smith',
 'J.R. Smith',
 'Joe Smith']

In [19]:
len([name for name in data[data["TARGET_5Yrs"]==1].Name.to_numpy() if re.search("chris", name, flags=re.IGNORECASE)])

21

In [20]:
len([name for name in data[data["TARGET_5Yrs"]==1].Name.to_numpy() if re.search("williams", name, flags=re.IGNORECASE)])

22

In [23]:
len([name for name in data[data["TARGET_5Yrs"]==1].Name.to_numpy() if re.search("johnson", name, flags=re.IGNORECASE)])

16

In [30]:
data[data["MIN"] == 42]

,Name,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs


In [34]:
data["FGM"].describe()

count    1340.000000
mean        2.629104
std         1.683555
min         0.300000
25%         1.400000
50%         2.100000
75%         3.400000
max        10.200000
Name: FGM, dtype: float64

In [32]:
data["MIN"].describe()

count    1340.000000
mean       17.624627
std         8.307964
min         3.100000
25%        10.875000
50%        16.100000
75%        22.900000
max        40.900000
Name: MIN, dtype: float64